- 1 测试jd cross
- 2 DCN DeepFM

In [4]:
import warnings
import pandas as pd
import numpy as np
import datetime
import os
import platform
from sklearn.preprocessing import OneHotEncoder,LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
import pickle

%matplotlib inline
warnings.filterwarnings("ignore")
pd.set_option('display.max_colwidth',100)

In [5]:
# load traindataset
with open("../output/traindataset.pkl", 'rb') as fo:     # 读取pkl文件数据
    traindataset = pickle.load(fo, encoding='bytes')

traindataset["ts"] = traindataset["ts"].apply(lambda x: pd.to_datetime(x))
print(traindataset.shape)
traindataset.head()

(695200, 48)


,ts,unit,qty,year,month,day,week,geography_level_1,geography_level_2,geography_level_3,...,geo1mean14,geo2mean14,geo3mean14,pro1mean14,pro2mean14,geo1median14,geo2median14,geo3median14,pro1median14,pro2median14
0,2018-06-04,386,11926.8286,2018,6,4,0,5,12,11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1100,2018-06-04,463,6282.7266,2018,6,4,0,0,3,75,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2200,2018-06-04,273,285.3290,2018,6,4,0,0,10,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3300,2018-06-04,520,6672.6452,2018,6,4,0,1,11,23,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4400,2018-06-04,480,15204.5902,2018,6,4,0,7,4,76,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
traindataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 695200 entries, 0 to 695199
Data columns (total 48 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   ts                 695200 non-null  datetime64[ns]
 1   unit               695200 non-null  int64         
 2   qty                345316 non-null  float64       
 3   year               695200 non-null  int64         
 4   month              695200 non-null  int64         
 5   day                695200 non-null  int64         
 6   week               695200 non-null  int64         
 7   geography_level_1  695200 non-null  int64         
 8   geography_level_2  695200 non-null  int64         
 9   geography_level_3  695200 non-null  int64         
 10  product_level_1    695200 non-null  int64         
 11  product_level_2    695200 non-null  int64         
 12  unit_pro           695200 non-null  int64         
 13  geo_pro            695200 non-null  int64   

# 1. 模型训练


- 1.JD_cross2
- 2.DeepFM
- ref  
https://github.com/PENGZhaoqing/TimeSeriesPrediction  
https://github.com/gabrielpreda/Kaggle/blob/master/SantanderCustomerTransactionPrediction/starter-code-saving-and-loading-lgb-xgb-cb.py

In [7]:
# features
sparse_features = ['unit', 'year', 'month', 'day', 'week', 'geography_level_1', 'geography_level_2', 'geography_level_3', 
                   'product_level_1', 'product_level_2', 'unit_pro', 'geo_pro']
dense_features = ['weight',
       'last14max', 'last14min', 'last14std', 'last14mean', 'last14median',
       'last14sum', 'last7max', 'last7min', 'last7std', 'last7mean',
       'last7median', 'last7sum', 'last3max', 'last3min', 'last3std',
       'last3mean', 'last3median', 'last3sum', 'last3value', 'last1value', 'last2mean',
       'last2sum', 'last2value', 'geo1mean14', 'geo2mean14', 'geo3mean14', 'pro1mean14',
       'pro2mean14', 'geo1median14', 'geo2median14', 'geo3median14',
       'pro1median14', 'pro2median14']

target = ['qty']

In [103]:
dataset = traindataset.copy()

In [104]:
## densce_feature = np.log 处理
# dense_log = [name for name in dense_features if name != 'weight']

# for name in dense_log:
#     dataset[name] = np.log(dataset[name].astype(float) + 1)

In [105]:
# traindataset[:100].loc[~traindataset[:100]['geo1mean14'].isnull()]
# traindataset[np.isnan(traindataset['qty'].values)]
# traindataset['qty'][np.isinf(traindataset['qty'])] = 0.0 

# 替换空值，和选择大于0的数据
dataset = dataset.dropna(axis=0, how='any')
print(dataset.shape)
print(np.isnan(dataset['qty']).any())
print(np.isinf(dataset['qty']).any())

(343425, 48)
False
False


In [106]:
dataset.head()

,ts,unit,qty,year,month,day,week,geography_level_1,geography_level_2,geography_level_3,...,geo1mean14,geo2mean14,geo3mean14,pro1mean14,pro2mean14,geo1median14,geo2median14,geo3median14,pro1median14,pro2median14
2,2018-06-06,386,11926.8286,2018,6,6,2,5,12,11,...,9322.3656,9322.36560,10257.857667,10866.343811,10826.978933,9402.9722,9402.9722,11574.9116,7465.9504,7368.8416
1102,2018-06-06,463,6316.9266,2018,6,6,2,0,3,75,...,11468.3274,6292.42660,6292.426600,10866.343811,10826.978933,9257.5354,6292.4266,6292.4266,7465.9504,7368.8416
2202,2018-06-06,273,290.9290,2018,6,6,2,0,10,7,...,11468.3274,12503.50756,12786.431900,10866.343811,10826.978933,9257.5354,12262.2442,12769.1319,7465.9504,7368.8416
3302,2018-06-06,520,6116.8452,2018,6,6,2,1,11,23,...,18160.9770,17557.10800,6594.245200,10866.343811,10826.978933,21302.7755,22596.5670,6594.2452,7465.9504,7368.8416
4402,2018-06-06,480,15212.3902,2018,6,6,2,7,4,76,...,11336.2203,11336.22030,11336.220300,10866.343811,10826.978933,11335.3203,11335.3203,11335.3203,7465.9504,7368.8416


In [107]:
dataset.columns

Index(['ts', 'unit', 'qty', 'year', 'month', 'day', 'week',
       'geography_level_1', 'geography_level_2', 'geography_level_3',
       'product_level_1', 'product_level_2', 'unit_pro', 'geo_pro', 'weight',
       'last14max', 'last14min', 'last14std', 'last14mean', 'last14median',
       'last14sum', 'last7max', 'last7min', 'last7std', 'last7mean',
       'last7median', 'last7sum', 'last3max', 'last3min', 'last3std',
       'last3mean', 'last3median', 'last3sum', 'last3value', 'last1value',
       'last2mean', 'last2sum', 'last2value', 'geo1mean14', 'geo2mean14',
       'geo3mean14', 'pro1mean14', 'pro2mean14', 'geo1median14',
       'geo2median14', 'geo3median14', 'pro1median14', 'pro2median14'],
      dtype='object')

## 1.1 JD_cross2 

## 1.2 DeepFM

In [125]:
def sparseFeature(feat, feat_num, embed_dim=4):
    """
    create dictionary for sparse feature
    :param feat: feature name
    :param feat_num: the total number of sparse features that do not repeat
    :param embed_dim: embedding dimension
    :return:
    """
    return {'feat': feat, 'feat_num': feat_num, 'embed_dim': embed_dim}


def denseFeature(feat):
    """
    create dictionary for dense feature
    :param feat: dense feature name
    :return:
    """
    return {'feat': feat}

In [126]:
# # 获取test, 便于还原
# test = dataset[dataset.ts > pd.to_datetime("20210301")]
# max_value = np.max(test['qty'].values)
# min_value = np.min(test['qty'].values)

# print(max_value, min_value)

In [154]:
embed_dim=32
test_size=0.2

# encoder done!!!
encoder = ['geography_level_1','geography_level_2','geography_level_3','product_level_1','product_level_2']
unit_all = [ 'unit_pro', 'geo_pro', 'unit']

# 1 Encode 
for feat in sparse_features:
    if feat not in encoder+unit_all:
        le = LabelEncoder()
        dataset[feat] = le.fit_transform(dataset[feat])

# 2 log10()
# mms = MinMaxScaler(feature_range=(0, 1))
# dataset[dense_features] = mms.fit_transform(dataset[dense_features])
dense_features_log = [name for name in dense_features if name != 'weight'] + target
for feat in dense_features_log:
    if feat == 'qty':
        dataset[feat] = np.log(dataset[feat].astype(float) + 1)
    # dataset[feat] = np.log10(dataset[feat].astype(float) + 1)
    dataset[feat] = dataset[feat].astype(float)

feature_columns = [[denseFeature(feat) for feat in dense_features]] + \
                  [[sparseFeature(feat, len(dataset[feat].unique()), embed_dim=embed_dim)
                    for feat in sparse_features]]

In [155]:
feature_columns

[[{'feat': 'weight'},
  {'feat': 'last14max'},
  {'feat': 'last14min'},
  {'feat': 'last14std'},
  {'feat': 'last14mean'},
  {'feat': 'last14median'},
  {'feat': 'last14sum'},
  {'feat': 'last7max'},
  {'feat': 'last7min'},
  {'feat': 'last7std'},
  {'feat': 'last7mean'},
  {'feat': 'last7median'},
  {'feat': 'last7sum'},
  {'feat': 'last3max'},
  {'feat': 'last3min'},
  {'feat': 'last3std'},
  {'feat': 'last3mean'},
  {'feat': 'last3median'},
  {'feat': 'last3sum'},
  {'feat': 'last3value'},
  {'feat': 'last1value'},
  {'feat': 'last2mean'},
  {'feat': 'last2sum'},
  {'feat': 'last2value'},
  {'feat': 'geo1mean14'},
  {'feat': 'geo2mean14'},
  {'feat': 'geo3mean14'},
  {'feat': 'pro1mean14'},
  {'feat': 'pro2mean14'},
  {'feat': 'geo1median14'},
  {'feat': 'geo2median14'},
  {'feat': 'geo3median14'},
  {'feat': 'pro1median14'},
  {'feat': 'pro2median14'}],
 [{'feat': 'unit', 'feat_num': 632, 'embed_dim': 32},
  {'feat': 'year', 'feat_num': 4, 'embed_dim': 32},
  {'feat': 'month', 'fea

In [156]:
# 数据集划分
train = dataset[dataset.ts <= pd.to_datetime("20210301")]
test = dataset[dataset.ts > pd.to_datetime("20210301")]

In [168]:
train

,ts,unit,qty,year,month,day,week,geography_level_1,geography_level_2,geography_level_3,...,geo1mean14,geo2mean14,geo3mean14,pro1mean14,pro2mean14,geo1median14,geo2median14,geo3median14,pro1median14,pro2median14
2,2018-06-06,386,1.206126,0,5,5,2,5,12,11,...,9322.365600,9322.365600,10257.857667,10866.343811,10826.978933,9402.972200,9402.972200,11574.911600,7465.950400,7368.841600
1102,2018-06-06,463,1.187046,0,5,5,2,0,3,75,...,11468.327400,6292.426600,6292.426600,10866.343811,10826.978933,9257.535400,6292.426600,6292.426600,7465.950400,7368.841600
2202,2018-06-06,273,1.064226,0,5,5,2,0,10,7,...,11468.327400,12503.507560,12786.431900,10866.343811,10826.978933,9257.535400,12262.244200,12769.131900,7465.950400,7368.841600
3302,2018-06-06,520,1.186037,0,5,5,2,1,11,23,...,18160.977000,17557.108000,6594.245200,10866.343811,10826.978933,21302.775500,22596.567000,6594.245200,7465.950400,7368.841600
4402,2018-06-06,480,1.213034,0,5,5,2,7,4,76,...,11336.220300,11336.220300,11336.220300,10866.343811,10826.978933,11335.320300,11335.320300,11335.320300,7465.950400,7368.841600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
690701,2021-03-01,302,1.096779,3,2,0,0,0,3,75,...,9615.112417,12070.701765,13556.944969,7594.989885,2795.366300,2784.529622,4174.833333,4981.666667,1698.166667,442.166667
691801,2021-03-01,351,0.022036,3,2,0,0,1,24,31,...,8800.525911,9400.703053,0.996365,2709.366586,885.418008,2163.780924,2375.538466,0.314670,946.840007,1027.680664
692901,2021-03-01,581,1.104744,3,2,0,0,1,24,1,...,8800.525911,9400.703053,13372.251816,492.584735,207.503153,2163.780924,2375.538466,2560.486654,107.227762,59.348614
694001,2021-03-01,230,1.145573,3,2,0,0,1,24,1,...,8800.525911,9400.703053,13372.251816,492.584735,777.666317,2163.780924,2375.538466,2560.486654,107.227762,500.000000


In [157]:
print('The shape of X_train:{}'.format(train.shape))
print('The shape of X_test:{}'.format(test.shape))

The shape of X_train:(281489, 48)
The shape of X_test:(61936, 48)


In [158]:
# 数据集划分
train_X = [train[dense_features].values, train[sparse_features].values.astype('int32')]
train_y = train[target].values.astype('int32')
test_X = [test[dense_features].values, test[sparse_features].values.astype('int32')]
test_y = test[target].values.astype('int32')

In [159]:
import tensorflow as tf
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import AUC
from tensorflow import keras
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import Embedding, Dropout, Dense, Input, Layer
import os



In [160]:
class FM(Layer):
	def __init__(self, k=10, w_reg=1e-4, v_reg=1e-4):
		"""
		Factorization Machine
		:param k: A scalar. The dimension of the latent vector.
		:param w_reg: A scalar. The regularization coefficient of parameter w.
		:param v_reg: A scalar. The regularization coefficient of parameter v.
		"""
		super(FM, self).__init__()
		self.k = k  # 定义隐藏层的层数
		self.w_reg = w_reg  # 参数 w 的正则化系数
		self.v_reg = v_reg  # 参数 v 的正则化系数

	def build(self, input_shape):
		self.w0 = self.add_weight(name='w0', shape=(1,),
								  initializer=tf.zeros_initializer(),
								  trainable=True)
		self.w = self.add_weight(name='w', shape=(input_shape[-1], 1),
								 initializer='random_uniform',
								 regularizer=l2(self.w_reg),
								 trainable=True)
		self.V = self.add_weight(name='V', shape=(self.k, input_shape[-1]),
								 initializer='random_uniform',
								 regularizer=l2(self.v_reg),
								 trainable=True)

	def call(self, inputs, **kwargs):
		# first order
		first_order = self.w0 + tf.matmul(inputs, self.w)
		# second order
		second_order = 0.5 * tf.reduce_sum(
			tf.pow(tf.matmul(inputs, tf.transpose(self.V)), 2) -
			tf.matmul(tf.pow(inputs, 2), tf.pow(tf.transpose(self.V), 2)), axis=1, keepdims=True)
		return first_order + second_order


class DNN(Layer):
	"""
	Deep part
	"""
	def __init__(self, hidden_units, activation='relu', dnn_dropout=0.):
		"""
		DNN part
		:param hidden_units: A list. List of hidden layer units's numbers
		:param activation: A string. Activation function
		:param dnn_dropout: A scalar. dropout number
		"""
		super(DNN, self).__init__()
		self.dnn_network = [Dense(units=unit, activation=activation) for unit in hidden_units]
		self.dropout = Dropout(dnn_dropout)

	def call(self, inputs, **kwargs):
		x = inputs
		for dnn in self.dnn_network:
			x = dnn(x)
		x = self.dropout(x)
		return x


class DeepFM(keras.Model):
	def __init__(self, feature_columns, k=10, hidden_units=(200, 200, 200), dnn_dropout=0.,
				 activation='relu', fm_w_reg=1e-4, fm_v_reg=1e-4, embed_reg=1e-4):
		"""
		DeepFM
		:param feature_columns: A list. a list containing dense and sparse column feature information.
		:param k: A scalar. fm's latent vector number.
		:param hidden_units: A list. A list of dnn hidden units.
		:param dnn_dropout: A scalar. Dropout of dnn.
		:param activation: A string. Activation function of dnn.
		:param fm_w_reg: A scalar. The regularizer of w in fm.
		:param fm_v_reg: A scalar. The regularizer of v in fm.
		:param embed_reg: A scalar. The regularizer of embedding.
		"""
		super(DeepFM, self).__init__()
		self.dense_feature_columns, self.sparse_feature_columns = feature_columns  # feature_columns是被分成了稠密和稀疏特征
		# 对稀疏特征sparse_feature_columns进行embedding处理，每一列转化成embedding==8的列向量
		self.embed_layers = {
			'embed_' + str(i): Embedding(input_dim=feat['feat_num'],
										 input_length=1,
										 output_dim=feat['embed_dim'],
										 embeddings_initializer='random_uniform',
										 embeddings_regularizer=l2(embed_reg))
			for i, feat in enumerate(self.sparse_feature_columns)
		}
		self.fm = FM(k, fm_w_reg, fm_v_reg)
		self.dnn = DNN(hidden_units, activation, dnn_dropout)
		self.dense = Dense(1, activation=None)

	def call(self, inputs, **kwargs):
		dense_inputs, sparse_inputs = inputs
		# sparse_inputs.shape[1] == 26 遍历26个“稀疏”特征，并将其转化成 embedding ==8 的这个特征的域，
		# 最后，将这些特征进行拼接  208 = 26*8
		sparse_embed = tf.concat([self.embed_layers['embed_{}'.format(i)](sparse_inputs[:, i])
                                  for i in range(sparse_inputs.shape[1])], axis=-1)   # shape=(None,208)
		stack = tf.concat([dense_inputs, sparse_embed], axis=-1)
		# wide
		wide_outputs = self.fm(stack)  # shape=(none,1)
		# deep
		deep_outputs = self.dnn(stack)  # shape=(none,64)
		deep_outputs = self.dense(deep_outputs)  # shape=(none,1)

		outputs = tf.nn.sigmoid(tf.add(wide_outputs, deep_outputs))
		return outputs

	def summary(self):
		dense_inputs = Input(shape=(len(self.dense_feature_columns),), dtype=tf.float32)
		sparse_inputs = Input(shape=(len(self.sparse_feature_columns),), dtype=tf.int32)
		keras.Model(inputs=[dense_inputs, sparse_inputs], outputs=self.call([dense_inputs, sparse_inputs])).summary()

In [161]:
# tf.test.is_gpu_available(cuda_only=True)

gpu = tf.config.experimental.list_physical_devices(device_type='GPU')
print(gpu)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:2', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:3', device_type='GPU')]


In [162]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [163]:
read_part = True
sample_num = 600000
test_size = 0.2

embed_dim = 32
k = 10
dnn_dropout = 0.1
hidden_units = [256, 128, 64]

learning_rate = 0.001
batch_size = 1000
epochs = 100


model = DeepFM(feature_columns, k=k, hidden_units=hidden_units, dnn_dropout=dnn_dropout)
model.summary()

Model: "model_11"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_24 (InputLayer)           [(None, 12)]         0                                            
__________________________________________________________________________________________________
tf.__operators__.getitem_132 (S (None,)              0           input_24[0][0]                   
__________________________________________________________________________________________________
tf.__operators__.getitem_133 (S (None,)              0           input_24[0][0]                   
__________________________________________________________________________________________________
tf.__operators__.getitem_134 (S (None,)              0           input_24[0][0]                   
___________________________________________________________________________________________

In [164]:
# 定义损失函数
def custom_loss(y_actual, y_pred):
    y_actual = tf.cast(y_actual,dtype=tf.float32)
    y_pred = tf.cast(y_pred,dtype=tf.float32)
    custom_loss= (y_actual - y_pred)**2
    return custom_loss

In [165]:
# ============================model checkpoint======================
check_path = '../output/deepfm/deepfm_weights.epoch_{epoch:04d}.val_loss_{val_loss:.4f}.ckpt'
checkpoint = tf.keras.callbacks.ModelCheckpoint(check_path, save_weights_only=True,
                                                verbose=1, period=5)

# ============================Compile============================

model.compile(loss='mse',
              optimizer='Adam',
              metrics=['mse'])
# ==============================Fit==============================
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_auc', min_delta=0.01,
                                                  patience=1, verbose=0, mode='max',
                                                  baseline=None, restore_best_weights=True)
model.fit(
    train_X,
    train_y,
    epochs=epochs,
    callbacks=[early_stopping],
    batch_size=batch_size,
    validation_split=0.1
)


Epoch 1/100
254/254 [==============================] - 4s 8ms/step - loss: 0.2510 - mse: 0.2506 - val_loss: 0.1322 - val_mse: 0.1321
Epoch 2/100
254/254 [==============================] - 2s 6ms/step - loss: 0.1471 - mse: 0.1471 - val_loss: 0.1321 - val_mse: 0.1321
Epoch 3/100
254/254 [==============================] - 2s 7ms/step - loss: 0.1471 - mse: 0.1471 - val_loss: 0.1321 - val_mse: 0.1321
Epoch 4/100
254/254 [==============================] - 2s 6ms/step - loss: 0.1471 - mse: 0.1471 - val_loss: 0.1321 - val_mse: 0.1321
Epoch 5/100
254/254 [==============================] - 2s 7ms/step - loss: 0.1471 - mse: 0.1471 - val_loss: 0.1321 - val_mse: 0.1321
Epoch 6/100
254/254 [==============================] - 2s 7ms/step - loss: 0.1471 - mse: 0.1471 - val_loss: 0.1321 - val_mse: 0.1321
Epoch 7/100
254/254 [==============================] - 2s 7ms/step - loss: 0.1471 - mse: 0.1471 - val_loss: 0.1321 - val_mse: 0.1321
Epoch 8/100
254/254 [==============================] - 2s 7ms/step - 

In [167]:
# ===========================Test==============================


NameError: name 'trian' is not defined

In [166]:
prediction = model.predict(test_X)
prediction = prediction[:, 0]
print(prediction)

[1. 1. 1. ... 1. 1. 1.]


In [48]:
test_X

[array([[7.90000000e-02, 4.19429308e+00, 4.19005222e+00, ...,
         3.75852080e+00, 4.22650943e+00, 4.22074205e+00],
        [2.05000000e-01, 4.61495968e+00, 4.60811857e+00, ...,
         3.69731654e+00, 4.22650943e+00, 4.22074205e+00],
        [1.00000000e-02, 3.25761355e+00, 3.24445748e+00, ...,
         3.85859734e+00, 4.22650943e+00, 4.22074205e+00],
        ...,
        [4.00000000e-03, 2.91860004e+00, 2.90024427e+00, ...,
         3.46725788e+00, 2.21583215e+00, 1.94734920e+00],
        [1.00000000e-02, 3.30133751e+00, 3.29067445e+00, ...,
         3.46725788e+00, 2.21583215e+00, 2.74024752e+00],
        [1.00000000e-03, 1.47712125e+00, 1.47712125e+00, ...,
         2.96951227e+00, 3.29531048e+00, 4.40021841e+00]]),
 array([[ 386,    3,    2, ...,    6,  761,   85],
        [ 463,    3,    2, ...,    6,  913,  518],
        [ 273,    3,    2, ...,    6,  537,   54],
        ...,
        [ 581,    3,    5, ...,   14, 1144,    9],
        [ 230,    3,    5, ...,   16,  451,   10

In [31]:

prediction = prediction * (max_value - min_value) + min_value
print(prediction)



[1.30069111e-28 0.00000000e+00 8.53590640e-28 ... 1.18498085e-11
 3.41056480e-12 9.22822027e-19]


In [112]:
test

,ts,unit,qty,year,month,day,week,geography_level_1,geography_level_2,geography_level_3,...,geo1mean14,geo2mean14,geo3mean14,pro1mean14,pro2mean14,geo1median14,geo2median14,geo3median14,pro1median14,pro2median14
1002,2021-03-02,386,0.082403,3,2,1,1,5,12,11,...,0.095559,0.191889,0.189043,0.942393,0.582141,0.017207,0.057115,0.157900,0.929982,0.591379
2102,2021-03-02,463,0.215954,3,2,1,1,0,3,75,...,0.261972,0.465002,0.332678,0.942393,0.582141,0.074577,0.154936,0.137140,0.929982,0.591379
3202,2021-03-02,273,0.009601,3,2,1,1,0,10,7,...,0.261972,0.459282,0.275656,0.942393,0.582141,0.074577,0.125657,0.198826,0.929982,0.591379
4302,2021-03-02,520,0.193414,3,2,1,1,1,11,23,...,0.239564,0.335753,0.346706,0.942393,0.582141,0.057175,0.081171,0.081557,0.929982,0.591379
5402,2021-03-02,480,0.088937,3,2,1,1,7,4,76,...,0.117522,0.167663,0.178408,0.942393,0.582141,0.032125,0.046260,0.090576,0.929982,0.591379
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
690799,2021-06-07,302,0.007711,3,5,6,0,0,3,75,...,0.276594,0.534210,0.397432,0.364165,0.100976,0.072710,0.196279,0.162815,0.105866,0.033127
691899,2021-06-07,351,0.000543,3,5,6,0,1,24,31,...,0.253345,0.375334,0.000647,0.114570,0.071417,0.067160,0.099023,0.000038,0.061226,0.099296
692999,2021-06-07,581,0.004315,3,5,6,0,1,24,1,...,0.253345,0.375334,0.321470,0.007917,0.009026,0.067160,0.099023,0.080732,0.005601,0.003116
694099,2021-06-07,230,0.010314,3,5,6,0,1,24,1,...,0.253345,0.375334,0.321470,0.007917,0.026522,0.067160,0.099023,0.080732,0.005601,0.019526


In [150]:
!nvidia-smi

Tue Jun 21 16:03:07 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.119.04   Driver Version: 450.119.04   CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P40           On   | 00000000:00:0B.0 Off |                    0 |
| N/A   35C    P0    49W / 250W |  22496MiB / 22919MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla P40           On   | 00000000:00:0C.0 Off |                    0 |
| N/A   

In [ ]:
!kill -9 12106

In [1]:
!nvidia-smi

Tue Jun 21 17:17:51 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.119.04   Driver Version: 450.119.04   CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P40           On   | 00000000:00:0B.0 Off |                    0 |
| N/A   34C    P0    49W / 250W |   1939MiB / 22919MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla P40           On   | 00000000:00:0C.0 Off |                    0 |
| N/A   

In [ ]:
!kill -9 6759